# Amazon Reviews - Prediction of Rating and Helpfulness (an NLP Use Case)
## Data Gathering and Cleaning

In this note book I have gathered Amazon review dataset stored in an Amazon S3 bucket, and did preliminary exploration and cleaning.

## Table of Contents
* [Data Gathering](#dg)
* [Missing Values](#mv)
* [Dataset with Verified Purchase Only](#dv)
* [Subsetting Data for ML Prototyping](#sd)

#### Setting up PySpark and Accessing S3 Bucket

In [5]:
# If you are running this notebook outside of AWS EMR, please uncomment the following lines and run
# from pyspark.context import SparkContext
# from pyspark.sql.context import SQLContext
# from pyspark.sql.session import SparkSession

# sc = SparkContext()
# sqlContext = SQLContext(sc)
# spark = SparkSession(sc)

In [6]:
# # Replace the content inside <> with your S3 credentials
# ACCESS_KEY = "<aws-access-key>"
# SECRET_KEY = "<aws-secret-key>"
# ENCODED_SECRET_KEY = SECRET_KEY.replace("/", "%2F")
# AWS_BUCKET_NAME = "amazon-reviews-pds"
# MOUNT_NAME = "mount_1"

# dbutils.fs.mount("s3a://%s:%s@%s" % (ACCESS_KEY, ENCODED_SECRET_KEY, AWS_BUCKET_NAME), "/mnt/%s" % MOUNT_NAME)
# display(dbutils.fs.ls("/mnt/%s" % MOUNT_NAME))


In [7]:
sqlContext

Out[ 2 ]: <pyspark.sql.context.HiveContext at 0x7f2ac9a6bf28>

<a id='dg'></a>
## Data Gathering

Amazon review dataset is spread across 46 tsv files in amazon S3 bucket `amazon-reviews-pds`. Each file corresponds to one product category. There are 5 additional multilingual datasets, which I am not going to use in this project. To see the list of files in `amazon-reviews-pds` S3 bucket, you can use the command `aws s3 ls s3://amazon-reviews-pds/tsv/` in AWS command line interface (CLI). Setting up AWSCLI is explained in [Getting Started with AWS S3 CLI](https://www.youtube.com/watch?v=WrVqrvIQRAI&t=925s).

In [10]:
from pyspark.sql.types import StructType, StructField
from pyspark.sql.types import DoubleType, IntegerType, StringType, DateType

# Define the schema of the dataframe to be created
schema = StructType([
      StructField('marketplace', StringType()),
      StructField('customer_id', StringType()),
      StructField('review_id', StringType()),
      StructField('product_id', StringType()),
      StructField('product_parent', StringType()),
      StructField('product_title', StringType()),
      StructField('product_category', StringType()),
      StructField('star_rating', IntegerType()),
      StructField('helpful_votes', IntegerType()),
      StructField('total_votes', IntegerType()),
      StructField('vine', StringType()),
      StructField('verified_purchase', StringType()),
      StructField('review_headline', StringType()),
      StructField('review_body', StringType()),
      StructField('review_date', DateType())
])

In [11]:
# Read Amazon review files from Amazon S3
review_df = (sqlContext.read.format('com.databricks.spark.csv')
             .schema(schema)
             .option("inferSchema", False)
             .option('delimiter', '\t')
             .option("header", True)
             .load("/mnt/mount_1/tsv/amazon_reviews_us*.gz" ))

In [12]:
review_df.limit(5).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,25933450,RJOVP071AVAJO,0439873800,84656342,There Was an Old Lady Who Swallowed a Shell!,Books,5,0,0,N,Y,Five Stars,I love it and so does my students!,2015-08-31
1,US,1801372,R1ORGBETCDW3AI,1623953553,729938122,I Saw a Friend,Books,5,0,0,N,Y,"Please buy ""I Saw a Friend""! Your children wil...",My wife and I ordered 2 books and gave them as...,2015-08-31
2,US,5782091,R7TNRFQAOUTX5,142151981X,678139048,"Black Lagoon, Vol. 6",Books,5,0,0,N,Y,Shipped fast.,Great book just like all the others in the ser...,2015-08-31
3,US,32715830,R2GANXKDIFZ6OI,014241543X,712432151,If I Stay,Books,5,0,0,N,N,Five Stars,So beautiful,2015-08-31
4,US,14005703,R2NYB6C3R8LVN6,1604600527,800572372,Stars 'N Strips Forever,Books,5,2,2,N,Y,Five Stars,Enjoyed the author's story and his quilts are ...,2015-08-31


In [13]:
review_df.printSchema()

root
-- marketplace: string (nullable = true)
-- customer_id: string (nullable = true)
-- review_id: string (nullable = true)
-- product_id: string (nullable = true)
-- product_parent: string (nullable = true)
-- product_title: string (nullable = true)
-- product_category: string (nullable = true)
-- star_rating: integer (nullable = true)
-- helpful_votes: integer (nullable = true)
-- total_votes: integer (nullable = true)
-- vine: string (nullable = true)
-- verified_purchase: string (nullable = true)
-- review_headline: string (nullable = true)
-- review_body: string (nullable = true)
-- review_date: date (nullable = true)

In [14]:
review_df.count()

Out[ 7 ]: 150962278

In [15]:
review_df.describe().toPandas()

,summary,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body
0,count,150962265,150962265,150962265,150962265,150962265,150962265,150960056,150960002,150960002,150960002,150960002,150960002,150959694,150943490
1,mean,None,2.85228448704825E7,None,1.1393217385666413E9,5.001996346301954E8,Infinity,None,4.19977887917622,1.9055346263177713,2.5379710911768534,None,None,Infinity,Infinity
2,stddev,None,1.5622928720229633E7,None,1.2364959580461092E9,2.886706759847555E8,NaN,None,1.2540875645435217,19.511701757363053,21.216056454215632,None,None,NaN,NaN
3,min,US,100000,R100007TERQ36I,0000000078,100000041,--In pursuit of the American dream,2002-08-07,1,0,0,N,N,"Our 2-year old son loves his ""big bed"" with t...",__________________________________<br />I mad...
4,max,US,9999999,RZZZZYOFYZ829,BT00IU6O8K,999999945,🌴 Vacation On The Beach,Wireless,5,47524,48362,Y,Y,🤹🏽‍♂️🎤Great product. Practice makes perfect. D...,🛅🚑🚚🚏🚙🚈🚘🚈🚘🚏🚘🚙🚎🚎🚈🎶🎶🎷🏁🏁🎷🎓👚👚🎓👢🎩👡👒👡👒👡🎩👚🎒🚹🔡🔚🔚📵🅾🔜📴🔜🆎🔜...


<a id='mv'></a>
## Missing Values
If you compare the output of count() and describe() funciton above, you notice that there are some missing values. Investigate these missing values.

In [17]:
# If you compare the output of count and describe funciton, 13 rows seem to have NULL in review_id field, which is supposed to be unique ID. Investigate these rows. 
review_df.filter("review_id is NULL").toPandas()

# In these 13 rows all columns have null value. Looks like a blank line is added at the end of some files

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
5,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
6,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
7,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [18]:
# Investigate product_category
review_df.groupBy('product_category').count().toPandas()

,product_category,count
0,If you have ever been through something and yo...,1
1,PC,6908551
2,Major Appliances,96901
3,Lawn and Garden,2557287
4,Kitchen,4880460
5,Home Improvement,2634537
6,Home Entertainment,705889
7,2014-01-09,1
8,Home,6221537
9,"\\""Red Spring Blossom Flowers with Yellow Bees...",1


In [19]:
# In product_category, some rows have values that seem to be dates or product_review. Investigate some of these rows.
review_df[review_df.product_category.isin("2014-01-09", "2011-02-18", "2009-03-08")].toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,13696877,R1TD4SVQCXLYYP,B004DU61SI,801335364,"""Clear See Through Snap on Hard Skin Shell Pro...",2011-02-18,None,None,None,None,None,None,None,None
1,US,45953978,R2WOQSOF7U5QTZ,B001JTG82U,121111587,"""GOgroove SonaWave HD 2.1 Stereo Sound System ...",2009-03-08,None,None,None,None,None,None,None,None
2,US,7384819,R3QLMYOI7Y4CFC,B001U4ZZPK,661176068,"""Blue Sea Systems BelowDeck Panels, Dual USB C...",2014-01-09,None,None,None,None,None,None,None,None


In [20]:
# value_count is not avialable in pyspark as in pandas
review_df.groupBy('star_rating').count().toPandas()

,star_rating,count
0,NaN,2276
1,1.0,12099424
2,3.0,12133772
3,5.0,93199322
4,4.0,26223155
5,2.0,7304329


In [21]:
review_df.groupBy('verified_purchase').count().toPandas()

,verified_purchase,count
0,None,2276
1,Y,117711788
2,N,33248214


In [22]:
review_df.groupBy('marketplace').count().toPandas()

,marketplace,count
0,US,150962278


In [23]:
review_df.groupBy('vine').count().toPandas()

,vine,count
0,None,2276
1,Y,666329
2,N,150293673


In [24]:
from pyspark.sql.functions import min, max

# Get the date range of Amazon reviews
min_date, max_date = review_df.select(min("review_date"), max("review_date")).first()


In [25]:
min_date, max_date

Out[ 17 ]: (datetime.date(1995, 6, 24), datetime.date(2015, 8, 31))

### Important Characteristics of Features in the Dataset
All the features in the dataset are self-explanatory except vine. Y in vine column indicates that review is written as a part of Amazon vine program. In this program Amazon invites the most trusted reviewers on Amazon to post opinions about new and pre-release items to help other customers to make informed purchase decisions ([Reference](https://www.amazon.com/gp/vine/help)).

The data set has 150,962,278 rows and 13 columns. The review spans from 24 Jun, 1995 to 31 Dec, 2015. Amzon products sold in this time period has an average rating of 4.20. The rating distribution is as follows: 
- 1 star - 8%
- 2 star - 5%
- 3 star - 8%
- 4 star - 17%
- 5 star - 62%

Among 150.1 million reviews, 78% are written by customers with verified purchases. There are 43 different product categories.

### Dealing with Missing Values

There are 13 rows which do not have primary key (review_id). As verified above, these rows have null in all the columns. This is probabally due to addition of blank lines in tsv files. 18,788 rows have null in review_body column. Since there is no way to impute missing review_body, these rows will be deleted. Similarly, ~2,000 rows have null values in column like star_rating, helpful_votes, total_votes, review_headline. Since we have more than 100 million data points, deleting these rows would be an appropriate choice of handling missing values.

<a id='dv'></a>
## Dataset with Verified Purchase Only
As mentioned above 22% of the reviews are written by customers who do not have verfied purchase. One of the major problem in online review platforms like Amazon is fake reviews; positive reviews by vendors to promote themselves or negative reviews by competitors. The probability of a review being fake significantly increases when it is written by someone without a verified purchase. Therefore, I have decided to include only reviews labelled as verified purchase for further analysis.

So subset the dataset of verified purchases only with all the rows having missing values in various columns removed.

In [29]:
verified_review_df = (review_df.filter("verified_purchase = 'Y'").filter("star_rating is not NULL")
                      .filter("review_date is not NULL").filter("helpful_votes is not NULL")
                      .filter("total_votes is not NULL").filter("review_headline is not NULL")
                      .filter("review_body is not NULL").filter("review_id is not NULL"))

In [30]:
verified_review_df.count()

Out[ 19 ]: 117698232

In [31]:
verified_review_df.describe().toPandas()

,summary,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body
0,count,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232,117698232
1,mean,None,2.745626275473029E7,None,1.1670151515350382E9,5.00229682844086E8,Infinity,None,4.2272728276835965,1.3132003546153523,1.723108831405386,None,None,Infinity,Infinity
2,stddev,None,1.5461387097796759E7,None,1.2419889320119085E9,2.887973198900909E8,NaN,None,1.232003216200563,15.863270160642374,17.10822389634329,None,None,NaN,NaN
3,min,US,100000,R100007TERQ36I,0000000868,100000041,--In pursuit of the American dream,Apparel,1,0,0,N,Y,"Our 2-year old son loves his ""big bed"" with t...",Count me in as yet another person who receive...
4,max,US,9999997,RZZZZYOFYZ829,BT00E0U25U,999999945,🌴 Vacation On The Beach,Wireless,5,47524,48362,Y,Y,🤹🏽‍♂️🎤Great product. Practice makes perfect. D...,🛅🚑🚚🚏🚙🚈🚘🚈🚘🚏🚘🚙🚎🚎🚈🎶🎶🎷🏁🏁🎷🎓👚👚🎓👢🎩👡👒👡👒👡🎩👚🎒🚹🔡🔚🔚📵🅾🔜📴🔜🆎🔜...


In [32]:
verified_review_df.groupBy('verified_purchase').count().toPandas()

,verified_purchase,count
0,Y,117698232


In [33]:
verified_review_df.groupBy('product_category').count().toPandas()

,product_category,count
0,PC,6048487
1,Major Appliances,68837
2,Lawn and Garden,2251611
3,Kitchen,4098858
4,Home Improvement,2360429
5,Home Entertainment,523906
6,Home,5544085
7,Wireless,7928614
8,Video,95722
9,Digital_Video_Games,124313


In [34]:
verified_review_df.groupBy('star_rating').count().toPandas()

,star_rating,count
0,1,8807280
1,3,9461197
2,5,73754723
3,4,20113994
4,2,5561038


<a id='sd'></a>
## Subsetting Data for ML Prototyping
Since we have a lot of data, I will subset only 0.5% of the data, which is still ~600,000 rows. This subset will be used for ML prototyping.

In [36]:
# Sample a fracation of review_df by setting withReplacement=False and setting random seed to 42
sample_frac = 0.005
verified_review_df_sample = verified_review_df.sample(False, sample_frac, 42)

In [37]:
verified_review_df_sample.cache()
# If you want to remove this dataframe from cache, use verified_review_df_sample.unpersist(). If you do not unpersist, no matter what is written into 
# verified_review_df_sample, it takes cached verified_review_df_sample when you call the dataframe

Out[ 25 ]: DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]

In [38]:
verified_review_df_sample.count()

Out[ 26 ]: 588391

In [39]:
verified_review_df_sample.limit(5).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,11423253,R245YIAVJK82ZL,812211637X,285887177,Punjabi C.L. Bible / Common Language Version,Books,5,0,0,N,Y,Five Stars,Quality product fast shipping.,2015-08-31
1,US,50657459,R14MGR5CZBRFHL,1476792496,626321879,After We Collided (The After Series),Books,5,0,0,N,Y,Five Stars,great book,2015-08-31
2,US,51327644,RYQWAH30JYSKT,0807522171,981557037,Burn Girl,Books,5,0,0,N,Y,resolve and hope that both adults and young re...,"A moving contemporary story of courage, resolv...",2015-08-31
3,US,49940982,RHZ3GL7AGNL74,073870671X,203609936,The Well Worn Path,Books,5,0,0,N,Y,Five Stars,Great looks brand new!,2015-08-31
4,US,10282895,RL0KPJMUIIMBQ,1500562289,951665740,Simple Blessings: Coloring Designs to Encourag...,Books,5,2,3,N,Y,Love!,"I just bought a bunch of these, didn't know th...",2015-08-31


In [40]:
verified_review_df_sample.describe().toPandas()

,summary,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body
0,count,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391
1,mean,None,2.7465038481502946E7,None,1.170502552465037E9,4.999421599403254E8,Infinity,None,4.2272060585563,1.3046324637868356,1.7185188080715035,None,None,2962.7842553191485,9.500395157142857E7
2,stddev,None,1.5473797421706567E7,None,1.2544947634086635E9,2.887240290752733E8,NaN,None,1.2320601916042477,12.024384754516312,13.469935187983486,None,None,13473.185796543983,4.353627945842858E8
3,min,US,10000113,R1000T87SSZFDW,0001713353,100002377,! Small S 1pc Teal (Blue/Green) 1pc Purple / P...,Apparel,1,0,0,N,Y,,
4,max,US,9999984,RZZZUGIC9N6F9,BT00DDVMVQ,9999986,�Gildan Adult Heavy Blend� Full-Zip Hooded Swe...,Wireless,5,2569,2925,Y,Y,😤😤😥😥😈😈,🙊🙊🙊


In [41]:
verified_review_df_sample.groupBy('star_rating').count().toPandas()

,star_rating,count
0,1,43967
1,3,47287
2,5,368764
3,4,100428
4,2,27945


### Write data subset to a file

In [43]:
verified_review_df_sample.coalesce(1).write.format('com.databricks.spark.csv').option('header', 'true').save('dbfs:/FileStore/review_df_sample/review_df_sample.csv')

In [44]:
# Check if the file is present in dbfs
dbutils.fs.ls("dbfs:/FileStore")

Out[ 31 ]: [FileInfo(path='dbfs:/FileStore/import-stage/', name='import-stage/', size=0),
 FileInfo(path='dbfs:/FileStore/plots/', name='plots/', size=0),
 FileInfo(path='dbfs:/FileStore/review_df_sample/', name='review_df_sample/', size=0)]

In [45]:
# Check if the file is properly written by reading the file and investigating the dataframe created. Compare the charactersticks of the dataframe created to 
# the above dataframe.
subset_df = (sqlContext.read.format('com.databricks.spark.csv')
             .schema(schema)
             .option("inferSchema", False)
             .option("header", True)
             .load("dbfs:/FileStore/review_df_sample/review_df_sample.csv" ))

In [46]:
subset_df.cache()


Out[ 33 ]: DataFrame[marketplace: string, customer_id: string, review_id: string, product_id: string, product_parent: string, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: date]

In [47]:
subset_df.is_cached

Out[ 34 ]: True

In [48]:
subset_df.count()

Out[ 35 ]: 588391

In [49]:
subset_df.limit(5).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,11423253,R245YIAVJK82ZL,812211637X,285887177,Punjabi C.L. Bible / Common Language Version,Books,5,0,0,N,Y,Five Stars,Quality product fast shipping.,2015-08-31
1,US,50657459,R14MGR5CZBRFHL,1476792496,626321879,After We Collided (The After Series),Books,5,0,0,N,Y,Five Stars,great book,2015-08-31
2,US,51327644,RYQWAH30JYSKT,0807522171,981557037,Burn Girl,Books,5,0,0,N,Y,resolve and hope that both adults and young re...,"A moving contemporary story of courage, resolv...",2015-08-31
3,US,49940982,RHZ3GL7AGNL74,073870671X,203609936,The Well Worn Path,Books,5,0,0,N,Y,Five Stars,Great looks brand new!,2015-08-31
4,US,10282895,RL0KPJMUIIMBQ,1500562289,951665740,Simple Blessings: Coloring Designs to Encourag...,Books,5,2,3,N,Y,Love!,"I just bought a bunch of these, didn't know th...",2015-08-31


In [50]:
subset_df.describe().toPandas()

summary,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body
count,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391,588391
mean,null,2.7465038481502946E7,null,1.170502552465037E9,4.999421599403254E8,Infinity,null,4.2272060585563,1.3046324637868356,1.7185188080715035,null,null,2962.7842553191485,9.500395157142857E7
stddev,null,1.5473797421706518E7,null,1.2544947634086623E9,2.8872402907527256E8,NaN,null,1.232060191604249,12.024384754516307,13.469935187983488,null,null,13473.185796543983,4.353627945842858E8
min,US,10000113,R1000T87SSZFDW,0001713353,100002377,! Small S 1pc Teal (Blue/Green) 1pc Purple / Pink Replacement Bands + 1pc Free Small Grey Band With Clasp for Fitbit FLEX Only /No tracker/ Wireless Activity Bracelet Sport Wristband Fit Bit Flex Bracelet Sport Arm Band Armband,Apparel,1,0,0,N,Y,!,! ! ! BEAUTIFUL and GREAT APP ! ! !
max,US,9999984,RZZZUGIC9N6F9,BT00DDVMVQ,9999986,�Gildan Adult Heavy Blend� Full-Zip Hooded Sweatshirt (Red) (X-Large),Wireless,5,2569,2925,Y,Y,😤😤😥😥😈😈,🙊🙊🙊


In [51]:
subset_df.groupby('star_rating').count().toPandas()

star_rating,count
1,43967
3,47287
5,368764
4,100428
2,27945


In [52]:
subset_df.printSchema()

root
-- marketplace: string (nullable = true)
-- customer_id: string (nullable = true)
-- review_id: string (nullable = true)
-- product_id: string (nullable = true)
-- product_parent: string (nullable = true)
-- product_title: string (nullable = true)
-- product_category: string (nullable = true)
-- star_rating: integer (nullable = true)
-- helpful_votes: integer (nullable = true)
-- total_votes: integer (nullable = true)
-- vine: string (nullable = true)
-- verified_purchase: string (nullable = true)
-- review_headline: string (nullable = true)
-- review_body: string (nullable = true)
-- review_date: date (nullable = true)

In [53]:
subset_df.groupBy('product_category').count().toPandas()

,product_category,count
0,PC,30243
1,Major Appliances,373
2,Lawn and Garden,11488
3,Kitchen,20650
4,Home Improvement,11827
5,Home Entertainment,2611
6,Home,27726
7,Wireless,39496
8,Video,430
9,Digital_Video_Games,548
